# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
This notebook extracts and processes the data in the table of postcodes from the wikipedia page at: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

We ignore unused postcodes and merge all neighbourhoods with the same postcode into a single row, and assign the suburb name to the neighbourhood where one the neighbourhood has not been assigned a separate name.


### Import the packages we'll use

In [20]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML

### Download the wiki page and convert into an object tree

In [21]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

wiki_page = requests.get(url).text
    
soup = BeautifulSoup(wiki_page,'lxml')

### Find the table part of the page and extract the headers
3.1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [22]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#print(My_table)

ths = My_table.findAll('th')
headers=[]
for th in ths:
    headers.append(th.text.strip())
headers

['Postcode', 'Borough', 'Neighbourhood']

### Now add the data
Find all of the table rows containing data and build up a dictionary of postcodes, boroughs and neighbourhoods, filtering and appending neighbourhoods to existing postcodes as we go.
* 3.2 Only process the cells that have an assigned borough. _Ignore cells with a borough that is Not assigned._
* 3.3 More than one neighborhood can exist in one postal code area. _For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table._
* 3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. _So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park._

In [23]:
# Find all of the table rows to process
rows = My_table.findAll('tr')
postcode_list={}
# lose the first row as that's the headers that we have already processed
rows=rows[1:]
for row in rows:
    tds = row.findAll('td')
    postcode = tds[0].text.strip()
    borough = tds[1].text.strip()
    neigh = tds[2].text.strip()
    
    #3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
    if (neigh == "Not assigned"):
        neigh = borough

    ## 3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if (borough != 'Not assigned'):
        postcode_dict = {
            "Postcode": postcode, 
            "Borough": borough, 
            "Neighbourhood": neigh
        }
        ## 3.3 If we already have an entry for this postcode, then concatenate the neighbourhoods and updating the existing entry
        if (postcode in postcode_list):
            concatenated_neigh = postcode_list[postcode]["Neighbourhood"] + ", " + neigh
            postcode_dict.update({"Neighbourhood": concatenated_neigh})
            postcode_list[postcode].update(postcode_dict)
        else:
            postcode_list[postcode] = postcode_dict

### Convert our dictionary of postcode data into a dataframe
Show the first 12 entries.

In [24]:
df = pd.DataFrame(list(postcode_list.values()), columns=headers)
df.set_index('Postcode')
df.head(12)

,Postcode,Borough,Neighbourhood
0,M5N,Central Toronto,Roselawn
1,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
2,M4J,East York,East Toronto
3,M4R,Central Toronto,North Toronto West
4,M1X,Scarborough,Upper Rouge
5,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade
6,M2R,North York,Willowdale West
7,M4A,North York,Victoria Village
8,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn"
9,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village"


# Now check the shape of the data frame
3.6 In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [25]:
print("The dataframe has {} rows.".format(df.shape[0]))

The dataframe has 103 rows.


### Obtain co-ordinates for the suburbs
* 4.2 Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [26]:
#use geocoder or csv

import csv

csv_url = 'https://cocl.us/Geospatial_data'

with requests.Session() as s:
    download = s.get(csv_url)

    decoded_content = download.content.decode('utf-8')

    reader = csv.reader(decoded_content.splitlines(), delimiter=',')
    csv_data = list(reader)
            
latlong_df=pd.DataFrame(csv_data[1:], columns=['Postcode', 'Latitude', 'Longitude'])
latlong_df.set_index('Postcode')
latlong_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.2169174
4,M1H,43.773136,-79.2394761


### Update our dataframe with the co-ordinates of each postcode
e.g. 'M4N' should have co-ordinates: ('43.7280205', '-79.3887901')

In [27]:
neighbourhoods = pd.merge(df, latlong_df, how='outer', on='Postcode')
#convert the lat/long strings to numeric
neighbourhoods[['Latitude', 'Longitude']] = neighbourhoods[['Latitude', 'Longitude']].apply(pd.to_numeric)
neighbourhoods.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5N,Central Toronto,Roselawn,43.711695,-79.416936
1,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
2,M4J,East York,East Toronto,43.685347,-79.338106
3,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
4,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
5,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
6,M2R,North York,Willowdale West,43.782736,-79.442259
7,M4A,North York,Victoria Village,43.725882,-79.315572
8,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013
9,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191


### Explore and cluster the neighborhoods in Toronto
Explore and cluster the neighborhoods in Toronto. 
You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:
* to add enough Markdown cells to explain what you decided to do and to report any observations you make.
* to generate maps to visualize your neighborhoods and how they cluster together.

#### Use geopy library to get the latitude and longitude values of Toronto.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>t_explorer</em>, as shown below.

Import all the libraries we will use

In [54]:
#Import Libraries
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed this yet
#import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [55]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [56]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [57]:
### Delve a bit deeper and look at boroughs with 'Toronto' in their name

In [58]:
### So let's slice the original dataframe and create a new dataframe of the 'Toronto' borough data.

In [59]:
subset_data = neighbourhoods[neighbourhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
subset_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5N,Central Toronto,Roselawn,43.711695,-79.416936
1,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
2,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
3,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
4,M4S,Central Toronto,Davisville,43.704324,-79.388790


Let's get the geographical coordinates of DownTown Toronto.

In [60]:
address = 'DownTown Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of DownTown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of DownTown Toronto are 43.655115, -79.380219.


As we did with all of Toronto, let's visualize the neighborhoods in Downtown Toronto.

In [61]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(subset_data['Latitude'], subset_data['Longitude'], subset_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [68]:
CLIENT_ID = 'SSH54FSXROYXILRRQFG4PQIFYZBKZ5PV0OEULGIJJS2TLBJN' # your Foursquare ID
CLIENT_SECRET = 'CWXT4OKPBGEKWULOJJ0E2FSFU5MDQJF5YDRD5ETB3DMOOOO0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentials:
CLIENT_ID: SSH54FSXROYXILRRQFG4PQIFYZBKZ5PV0OEULGIJJS2TLBJN
CLIENT_SECRET:CWXT4OKPBGEKWULOJJ0E2FSFU5MDQJF5YDRD5ETB3DMOOOO0


In [63]:
subset_data.loc[0, 'Neighbourhood']
neighborhood_latitude = subset_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = subset_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = subset_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Roselawn are 43.7116948, -79.41693559999999.


#### Now, let's get the top 100 venues that are in Roselawn within a radius of 500 meters.

First, let's create the GET request URL.

In [64]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Send the GET request and examine the results

In [65]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9a91821ed219272c062b4a'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4e6e176c45dd293273b74e3c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/garden_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d15a941735',
         'name': 'Garden',
         'pluralName': 'Gardens',
         'primary': True,
         'shortName': 'Garden'}],
       'id': '4e6e176c45dd293273b74e3c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 402,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.71218888050602,
          'lng': -79.41197784736922}],
        'lat': 43.71218888050602,
        'lng

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [66]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [67]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rosalind's Garden Oasis,Garden,43.712189,-79.411978
